In [1]:
from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

import os
import numpy as np
import pandas as pd
from PIL import Image

SEED = 42

csv_path, csv_test_path = "../data/train.csv", "../data/test.csv"
img_dir, img_dir_test = "../data/train_ims", "../data/test_ims"

data_train = pd.read_csv(csv_path)
print(data_train)
data_test = pd.read_csv(csv_test_path)

X_train, y_train, X_test, y_test = [], [], [], []

for _, row in data_train.iterrows():
    img_path = os.path.join(img_dir, row.iloc[0])
    label = int(row.iloc[1])
    img = Image.open(img_path).convert("RGB")
    img = np.array(img).flatten()
    X_train.append(img)
    y_train.append(label)

for _, row in data_test.iterrows():
    img_path = os.path.join(img_dir_test, row.iloc[0])
    label = int(row.iloc[1])
    img = Image.open(img_path).convert("RGB")
    img = np.array(img).flatten()
    X_test.append(img)
    y_test.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED)


X_train_featured = np.load("../data/nparrays/X_train_features4.npy")
X_val_featured = np.load("../data/nparrays/X_val_features4.npy")

svm_model = make_pipeline(
    StandardScaler(), 
    PCA(n_components=0.75, random_state=SEED), 
    SVC(C=8, kernel='rbf', gamma='scale',random_state=SEED))

svm_model.fit(X_train_featured, y_train)

y_pred_svm = svm_model.predict(X_val_featured)
accuracy_svm = accuracy_score(y_val, y_pred_svm)
print(f"SVM Validation Accuracy: {accuracy_svm:.4f}")

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

# Classification Report 
print("Classification Report:")
print(classification_report(y_val, y_pred_svm))

           im_name  label
0      000231c.jpg      2
1      0002574.jpg      5
2      00027d5.jpg      7
3      000304e.jpg      3
4      00047fc.jpg      7
...            ...    ...
49995  d507197.jpg      5
49996  d508429.jpg      1
49997  d508cb7.jpg      9
49998  d509167.jpg      3
49999  d509c42.jpg      3

[50000 rows x 2 columns]
SVM Validation Accuracy: 0.7535
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.82      0.80      1011
           1       0.88      0.86      0.87      1051
           2       0.64      0.65      0.65       985
           3       0.55      0.56      0.55       983
           4       0.68      0.72      0.70       968
           5       0.64      0.61      0.63       975
           6       0.82      0.80      0.81      1022
           7       0.84      0.79      0.82       987
           8       0.85      0.86      0.85       996
           9       0.85      0.83      0.84      1022

    accurac